In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from bayes_opt import BayesianOptimization
import logging
import os
import time
from flask import Flask, request, jsonify


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(_name_)


stock_symbols = ['AAPL', 'MSFT', 'GOOGL']  # You can change this to any stock symbols
start_date = '2010-01-01'
end_date = '2023-01-01'

data = {}
for symbol in stock_symbols:
    data[symbol] = yf.download(symbol, start=start_date, end=end_date)


scalers = {}
for symbol in stock_symbols:
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data[symbol]['Close'].values.reshape(-1, 1))
    scalers[symbol] = scaler
    data[symbol]['Scaled Close'] = scaled_data


sequence_length = 60
X_train_all = {}
y_train_all = {}

for symbol in stock_symbols:
    X_train = []
    y_train = []
    for i in range(sequence_length, len(data[symbol])):
        X_train.append(data[symbol]['Scaled Close'][i-sequence_length:i])
        y_train.append(data[symbol]['Scaled Close'][i])
    X_train_all[symbol] = np.array(X_train)
    y_train_all[symbol] = np.array(y_train)


models = {}
for symbol in stock_symbols:
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=True, input_shape=(X_train_all[symbol].shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=100, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=100))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='loss', patience=5, verbose=1)
    model.fit(X_train_all[symbol], y_train_all[symbol], epochs=25, batch_size=32, callbacks=[early_stopping], verbose=2)
    
    models[symbol] = model

predictions = {}
for symbol in stock_symbols:
    predictions[symbol] = models[symbol].predict(X_train_all[symbol])
    predictions[symbol] = scalers[symbol].inverse_transform(predictions[symbol])


sns.set(style="whitegrid")
plt.figure(figsize=(14, 7))
for symbol in stock_symbols:
    plt.plot(data[symbol].index[sequence_length:], data[symbol]['Close'][sequence_length:], label=f'Actual Stock Price ({symbol})')
    plt.plot(data[symbol].index[sequence_length:], predictions[symbol], label=f'Predicted Stock Price ({symbol})')
plt.title('Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

app = Flask(_name_)

@app.route('/predict', methods=['POST'])
def predict():
    try:
        request_data = request.json
        symbol = request_data['symbol']
        X_test = request_data['X_test']
        model = models[symbol]
        scaler = scalers[symbol]
        if model is not None:
            predictions = model.predict(np.array(X_test).reshape(1, -1, 1))
            predictions = scaler.inverse_transform(predictions)
            return jsonify({'predictions': predictions.tolist()})
        else:
            return jsonify({'error': f"Model not found for {symbol}"})
    except Exception as e:
        logger.error(f"Error predicting for symbol {symbol}: {e}")
        return jsonify({'error': f"Error predicting for symbol {symbol}: {e}"}), 500

if _name_ == "_main_":
    app.run(debug=True)